# Prototyping and noodles

In [733]:
import cbpro
import pandas as pd
import pickle as pkl
import datetime
import matplotlib.pyplot as plt
import financialFunctions as ff
import coinbaseFunctions as cf
import utilityFunctions as uf
import numpy as np
import matplotlib.patches as patches
from decimal import *
import os
from os import listdir
from os.path import isfile, join
import Strategies as sg

#Makes it so a module is changed, it updates automatically on the code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [555]:
with open('../../../fredinhouTradingBot_Pvt/coinbase_credentials_1.pkl', 'rb') as f:
    credentials = pkl.load(f)
Client = cbpro.AuthenticatedClient(
    credentials['APIKey'],
    credentials['SecretKey'],
    credentials['passPhrase'],
    api_url = credentials['APIurl']
    )
testingDates = [[datetime.datetime.now() - datetime.timedelta(days = 3*356 + int(r)), datetime.datetime.now() - datetime.timedelta(days = 3*356 + int(r) - 60)] for r in np.random.random(size = 10)*120]

In [549]:
testingDates

[[datetime.datetime(2020, 2, 20, 16, 19, 38, 419939),
  datetime.datetime(2020, 4, 20, 16, 19, 38, 419949)],
 [datetime.datetime(2020, 3, 6, 16, 19, 38, 419952),
  datetime.datetime(2020, 5, 5, 16, 19, 38, 419953)],
 [datetime.datetime(2020, 4, 4, 16, 19, 38, 419955),
  datetime.datetime(2020, 6, 3, 16, 19, 38, 419956)],
 [datetime.datetime(2020, 3, 20, 16, 19, 38, 419958),
  datetime.datetime(2020, 5, 19, 16, 19, 38, 419959)],
 [datetime.datetime(2020, 2, 9, 16, 19, 38, 419961),
  datetime.datetime(2020, 4, 9, 16, 19, 38, 419962)],
 [datetime.datetime(2020, 2, 18, 16, 19, 38, 419964),
  datetime.datetime(2020, 4, 18, 16, 19, 38, 419965)],
 [datetime.datetime(2020, 1, 4, 16, 19, 38, 419967),
  datetime.datetime(2020, 3, 4, 16, 19, 38, 419968)],
 [datetime.datetime(2020, 4, 1, 16, 19, 38, 419969),
  datetime.datetime(2020, 5, 31, 16, 19, 38, 419971)],
 [datetime.datetime(2020, 3, 10, 16, 19, 38, 419972),
  datetime.datetime(2020, 5, 9, 16, 19, 38, 419973)],
 [datetime.datetime(2020, 3, 

In [472]:
cf.getData(Client, 'BTC-USD', start = testingDates[1][0], end = testingDates[1][1], granularity = 60, verbose = True, header = '')

All seems to be fine, gathering data from files.


,Close,High,Low,Open,Return,Volume
Date,,,,,,
2020-01-12 11:08:00,8088.05,8088.05,8084.07,8084.07,NaN,5.754785
2020-01-12 11:09:00,8085.29,8091.18,8085.29,8089.02,NaN,1.865473
2020-01-12 11:10:00,8082.49,8084.09,8081.83,8084.09,NaN,2.196877
2020-01-12 11:11:00,8075.02,8082.49,8075.01,8082.49,NaN,6.272719
2020-01-12 11:12:00,8077.28,8078.64,8075.01,8075.01,NaN,1.249073
...,...,...,...,...,...,...
2020-03-12 11:03:00,6146.66,6200.00,6138.73,6172.37,NaN,105.775310
2020-03-12 11:04:00,6113.61,6157.33,6110.37,6144.80,NaN,83.843506
2020-03-12 11:05:00,6084.04,6113.51,6063.18,6113.51,NaN,148.169720


In [557]:
assets = ['BTC-USD','ETH-USD','USD-USD']
ws = np.repeat(1/len(assets), len(assets))
for date in testingDates:
    dataDict = {}
    for asset in assets:
        dataDict[asset] = cf.getData(Client, asset, start = date[0], end = date[1], granularity = 60, verbose = False, header = '')[date[0] + datetime.timedelta(days = 15):date[0] + datetime.timedelta(days = 15, hours = 1)]
        if 'Return' in dataDict[asset].columns:
            dataDict[asset] = dataDict[asset].drop('Return', axis = 1)
        dataDict[asset]['Return'] = dataDict[asset]['Close'].pct_change()
        dataDict[asset] = dataDict[asset].dropna()
        
    params = {}
    params['dataDict'] = dataDict.copy()
    params['lookBackTime'] = 30*24*60
    params['minimumTrade'] = 5.0
    params['currentPortfolio'] = {assets[i]:ws[i] for i in range(len(assets))}
    params['totalFunds'] = 100.0
    print("Old Best Portfolio: " + str(Strategies.maxSharpeRatio(params)[2]))
    print("New Best Portfolio: " + str(Strategies.maxLogSigRatio(params)[2]))
    print('----------------------------------------------------------------')
    

Old Best Portfolio: {'BTC-USD': 1.0, 'ETH-USD': 0.0, 'USD-USD': 0.0}
New Best Portfolio: {'BTC-USD': 1.0, 'ETH-USD': 0.0, 'USD-USD': 0.0}
----------------------------------------------------------------
Old Best Portfolio: {'BTC-USD': 0.2915306975258765, 'ETH-USD': 0.6280555294127702, 'USD-USD': 0.08041377306135325}
New Best Portfolio: {'BTC-USD': 0.0, 'ETH-USD': 1.0, 'USD-USD': 0.0}
----------------------------------------------------------------
Old Best Portfolio: {'BTC-USD': 1.0, 'ETH-USD': 0.0, 'USD-USD': 0.0}
New Best Portfolio: {'BTC-USD': 1.0, 'ETH-USD': 0.0, 'USD-USD': 0.0}
----------------------------------------------------------------
Old Best Portfolio: {'BTC-USD': 1.0, 'ETH-USD': 0.0, 'USD-USD': 0.0}
New Best Portfolio: {'BTC-USD': 1.0, 'ETH-USD': 0.0, 'USD-USD': 0.0}
----------------------------------------------------------------
Old Best Portfolio: {'BTC-USD': 0.0, 'ETH-USD': 1.0, 'USD-USD': 0.0}
New Best Portfolio: {'BTC-USD': 0.0, 'ETH-USD': 0.4963023338721427, 'USD-

In [540]:
testingDates[7]

[datetime.datetime(2020, 3, 6, 15, 50, 31, 387826),
 datetime.datetime(2020, 5, 5, 15, 50, 31, 387827)]

In [546]:
dataDict

{'BTC-USD':                        Close     High      Low     Open  Return     Volume
 Date                                                                      
 2020-03-21 15:51:00  6364.48  6381.70  6352.06  6374.72     NaN  90.780039
 2020-03-21 15:52:00  6352.01  6368.26  6352.00  6364.47     NaN  40.521722
 2020-03-21 15:53:00  6367.61  6367.89  6348.33  6352.75     NaN  13.572048
 2020-03-21 15:54:00  6358.60  6370.00  6353.74  6367.70     NaN  20.081588
 2020-03-21 15:55:00  6336.57  6361.93  6334.19  6358.60     NaN  32.144832
 ...                      ...      ...      ...      ...     ...        ...
 2020-04-20 15:46:00  6875.48  6877.04  6874.94  6874.95     NaN  11.348125
 2020-04-20 15:47:00  6875.85  6878.27  6874.94  6877.05     NaN  12.527635
 2020-04-20 15:48:00  6861.00  6876.24  6861.00  6875.85     NaN  18.913455
 2020-04-20 15:49:00  6866.85  6866.85  6856.91  6861.00     NaN  13.499915
 2020-04-20 15:50:00  6869.25  6869.63  6864.32  6865.34     NaN   6.200951
 

In [566]:
test = 'BTC-USD_1490417713_1493009713_60.pkl'
test.split("_",2)

['BTC-USD', '1490417713', '1493009713_60.pkl']

In [378]:
dataDict['BTC-USD'] = cf.getData(Client, 'BTC-USD', start = date[0], end = date[1], granularity = 60, verbose = False, header = '')
dataDict['ETH-USD'] = cf.getData(Client, 'BCH-USD', start = date[0], end = date[1], granularity = 60, verbose = False, header = '')
dataDict['ETH-USD'] = cf.getData(Client, 'ETH-USD', start = date[0], end = date[1], granularity = 60, verbose = False, header = '')
(dataDict['ETH-USD'].drop_duplicates()).index.duplicated().any()
#dataDict['BTC-USD']['Return'] = dataDict['BTC-USD']['Close'].pct_change().dropna()
#dataDict['BCH-USD']['Return'] = dataDict['BCH-USD']['Close'].pct_change().dropna()
#dataDict['ETH-USD']['Return'] = dataDict['ETH-USD']['Close'].pct_change().dropna()

False

In [329]:
xs = []
ys = []
for w1 in np.linspace(0,1,5):
    print(w1)
    for w2 in np.linspace(0,1,5):
        for w3 in np.linspace(0,1,5):
            ws = np.array([w1, w2, w3])
            ws = ws/sum(ws)
            i = 0
            port = {}
            for key in dataDict.keys():
                port[key] = ws[i]
                i += 1
            xs += [ff.portfolioVolatility(dataDict, port)]
            ys += [ff.portfolioReturn(dataDict, port)]
msrPort = ff.maximumSharpeRatio(dataDict)
msrX = ff.portfolioVolatility(dataDict, msrPort)
msrY = ff.portfolioReturn(dataDict, msrPort)


0.0


/Users/andreguimaraes/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


0.25
0.5
0.75
1.0


In [724]:
#Backtracking
downloadedAssets = ['AAVE-USD', 'ADA-USD', 'ALGO-USD', 'ATOM-USD', 'BAL-USD', 'BAND-USD', 'BCH-USD', 'BNT-USD', 'BTC-USD', 'CGLD-USD', 'COMP-USD', 'DASH-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'FIL-USD', 'GRT-USD', 'KNC-USD', 'LINK-USD', 'LRC-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NMR-USD', 'NU-USD', 'OMG-USD', 'OXT-USD', 'REN-USD', 'REP-USD', 'SKL-USD', 'SNX-USD', 'SUSHI-USD', 'UMA-USD', 'UNI-USD','WBTC-USD', 'XLM-USD', 'XTZ-USD', 'YFI-USD', 'ZEC-USD', 'ZRX-USD']
now = datetime.datetime.now()
testingTimeInterval = [now - datetime.timedelta(days = 3*365), now - datetime.timedelta(days = 1*365)]
epochDuration = 30*24*60*60
numberOfEpochs = 10
maxTimeDelta = (testingTimeInterval[-1]-testingTimeInterval[0]).total_seconds() - 3*epochDuration
testTimes = []
intervalBetweenDataGather = 60*60
for i in range(numberOfEpochs):
    t1 = testingTimeInterval[0] + datetime.timedelta(seconds = np.random.randint(low = epochDuration,high = maxTimeDelta))
    t2 = t1 + datetime.timedelta(seconds = int(2.1*epochDuration))
    testTimes += [[t1,t2]]

In [738]:
fundsPerTime = {}
i = 0.
timeStart1 = datetime.datetime.now()
for times in testTimes:
    timeStart2 = datetime.datetime.now()
    pct1 = i/len(testTimes)*100.
    header1 = uf.progressBar(pct1, time0 = timeStart1) + "\n"
    fundsPerTime[str(times)] = []
    goodAssets = []
    for asset in downloadedAssets:
        if ff.availableData(asset, times[0]) and availableData(asset, times[1]):
            goodAssets += [asset]
    goodAssets = goodAssets + ['USD-USD']
    tradingFeePct = 0.005
    currentPortfolioSize = {asset:0. for asset in goodAssets}
    currentPortfolioSize['USD-USD'] = 100.
    dataDict = cf.getData(Client, goodAssets, start = times[0], end = times[1], granularity = 60, verbose = False, header = '')
    currentDate = times[0] + datetime.timedelta(seconds = epochDuration)
    iniDiff = (currentDate - times[0]).total_seconds()
    while (currentDate - times[0]).total_seconds() < 2*epochDuration:
        pct2 = ((currentDate - times[0]).total_seconds()-iniDiff)/(2*epochDuration)*100.
        uf.clear()
        print(uf.progressBar(pct2, time0 = timeStart2, header = header1))
        dataDictSoFar = {}
        for key in dataDict.keys():
            dataDictSoFar[key] = dataDict[key][:currentDate]
        totalFunds = 0
        for key in currentPortfolioSize.keys():
            totalFunds += currentPortfolioSize[key]*dataDictSoFar[key]['Close'][-1]
        print(totalFunds)
        currentPortfolio = {}
        for key in currentPortfolioSize.keys():
            currentPortfolio[key] = currentPortfolioSize[key]*dataDictSoFar[key]['Close'][-1]/totalFunds
        
        params = {}
        params['dataDict'] = dataDictSoFar.copy()
        params['lookBackTime'] = 60*60*24
        params['minimumTrade'] = 5
        #The current portfolio when this was called.
        params['currentPortfolio'] = currentPortfolio
        #The total ammount of funds available (in USD)
        params['totalFunds'] = totalFunds
        buys, sells, bestPortfolio = sg.maxSharpeRatio(params)
        for key in buys.keys():
            buys[key] = buys[key]*totalFunds/(dataDictSoFar[key]['Close'][-1])
        for key in sells.keys():
            sells[key] = sells[key]*totalFunds/(dataDictSoFar[key]['Close'][-1])
        currentPortfolioSize = makeFakeTrades(buys, sells, currentPortfolioSize, dataDictSoFar, tradingFeePct)        
        currentDate += datetime.timedelta(seconds = intervalBetweenDataGather)
        fundsPerTime[str(times)] += [totalFunds]
    i += 1

[#.................................................] 
[####..............................................] (approx 53m 34s)
98.9274931337584


KeyboardInterrupt: 

In [726]:
dataDict['REP-USD']

,Close,High,Low,Open,Return,Volume,index
Date,,,,,,,
2019-10-23 10:21:00,7.57,7.57,7.42,7.42,0.000000,52.196472,4259.0
2019-10-23 10:22:00,7.41,7.42,7.41,7.42,-0.021136,271.000000,4260.0
2019-10-23 10:23:00,7.46,7.47,7.46,7.47,0.006748,16.600000,4261.0
2019-10-23 10:24:00,7.45,7.45,7.45,7.45,-0.001340,7.000000,4262.0
2019-10-23 10:25:00,7.45,7.45,7.45,7.45,0.000000,7.200000,4263.0
...,...,...,...,...,...,...,...
2019-12-25 09:52:00,9.34,9.34,9.34,9.34,0.000000,13.656073,5412.0
2019-12-25 10:05:00,9.30,9.30,9.30,9.30,-0.004283,0.494108,5413.0
2019-12-25 10:07:00,9.34,9.34,9.34,9.34,0.004301,17.833643,5414.0


In [720]:
def makeFakeTrades(buys, sells, portfolioSize, dataDict, tradingFeePct):
    for key in sells.keys():
        if key != 'USD-USD':
            portfolioSize[key] -= sells[key]
            portfolioSize['USD-USD'] += sells[key]*dataDict[key]['Close'][-1]*(1-tradingFeePct)
            
    for key in buys.keys():
        if key != 'USD-USD':
            portfolioSize[key] += buys[key]
            portfolioSize['USD-USD'] -= buys[key]*dataDict[key]['Close'][-1]*(1-tradingFeePct)
            
    return portfolioSize

In [671]:
with open("../candlesDataBase/ALGO-USD/ALGO-USD_1570733820_1573325040_60.pkl", 'rb') as f:
    data = pkl.load(f)
data[testTimes[-1][0]:testTimes[-1][1]]

,low,high,open,close,volume
time,,,,,
2019-10-24 00:58:00,0.2028,0.2028,0.2028,0.2028,1250
2019-10-24 01:22:00,0.2016,0.2016,0.2016,0.2016,3000
2019-10-24 01:35:00,0.2007,0.2007,0.2007,0.2007,291
2019-10-24 01:36:00,0.2006,0.2006,0.2006,0.2006,1515
2019-10-24 01:44:00,0.2005,0.2005,0.2005,0.2005,695
...,...,...,...,...,...
2019-11-09 12:34:00,0.2559,0.2559,0.2559,0.2559,1
2019-11-09 12:38:00,0.2561,0.2563,0.2563,0.2561,78
2019-11-09 12:42:00,0.2567,0.2589,0.2567,0.2589,6462


In [669]:
print((datetime.datetime.timestamp(testTimes[-1][0]), datetime.datetime.timestamp(testTimes[-1][1])))

(1571896636.904792, 1577084236.904792)


In [190]:
data.reset_index(inplace = True)

In [191]:
dataff.maximumSharpeRatio(dataDict)

,Date,Low,High,Open,Close,Volume,Return


In [244]:
with open("../candlesDataBase/ETH-USD/ETH-USD_1498231858_1500823858_60.pkl", 'rb') as f:
    print(pkl.load(f))

                      Close    High     Low    Open  Return      Volume
Date                                                                   
2017-06-23 10:31:00  328.00  328.07  328.00  328.07     NaN    7.869500
2017-06-23 10:32:00  328.06  328.06  328.00  328.05     NaN    7.497429
2017-06-23 10:33:00  328.00  328.06  328.00  328.01     NaN   33.493148
2017-06-23 10:34:00  328.00  328.06  328.00  328.01     NaN   19.979300
2017-06-23 10:35:00  328.02  328.02  328.00  328.01     NaN    9.417400
...                     ...     ...     ...     ...     ...         ...
2017-07-23 10:26:00  229.64  229.64  229.63  229.64     NaN    7.207176
2017-07-23 10:27:00  229.63  229.64  229.63  229.64     NaN   13.206300
2017-07-23 10:28:00  229.63  229.64  229.63  229.64     NaN   40.695905
2017-07-23 10:29:00  229.51  229.63  229.50  229.63     NaN  240.878183
2017-07-23 10:30:00  228.81  229.50  228.81  229.50     NaN  112.427434

[42996 rows x 6 columns]


In [465]:
df = pd.DataFrame([[1,1],[1,2],[2,1]])
df

,0,1
0,1,1
1,1,2
2,2,1


In [767]:
import itertools
numbers = np.linspace(0.,1., num = 11, endpoint=True)
numbers = np.round(numbers,3)
print(numbers)
result = [list(itertools.permutations(seq)) for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if (sum(seq) == 1 and len(seq) == 3)]
resultTemp = []
for res in result:
    for r in res:
        resultTemp += [r]
print(sorted(resultTemp))

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[(0.0, 0.1, 0.9), (0.0, 0.2, 0.8), (0.0, 0.3, 0.7), (0.0, 0.4, 0.6), (0.0, 0.6, 0.4), (0.0, 0.7, 0.3), (0.0, 0.8, 0.2), (0.0, 0.9, 0.1), (0.1, 0.0, 0.9), (0.1, 0.2, 0.7), (0.1, 0.3, 0.6), (0.1, 0.4, 0.5), (0.1, 0.5, 0.4), (0.1, 0.6, 0.3), (0.1, 0.7, 0.2), (0.1, 0.9, 0.0), (0.2, 0.0, 0.8), (0.2, 0.1, 0.7), (0.2, 0.3, 0.5), (0.2, 0.5, 0.3), (0.2, 0.7, 0.1), (0.2, 0.8, 0.0), (0.3, 0.0, 0.7), (0.3, 0.1, 0.6), (0.3, 0.2, 0.5), (0.3, 0.5, 0.2), (0.3, 0.6, 0.1), (0.3, 0.7, 0.0), (0.4, 0.0, 0.6), (0.4, 0.1, 0.5), (0.4, 0.5, 0.1), (0.4, 0.6, 0.0), (0.5, 0.1, 0.4), (0.5, 0.2, 0.3), (0.5, 0.3, 0.2), (0.5, 0.4, 0.1), (0.6, 0.0, 0.4), (0.6, 0.1, 0.3), (0.6, 0.3, 0.1), (0.6, 0.4, 0.0), (0.7, 0.0, 0.3), (0.7, 0.1, 0.2), (0.7, 0.2, 0.1), (0.7, 0.3, 0.0), (0.8, 0.0, 0.2), (0.8, 0.2, 0.0), (0.9, 0.0, 0.1), (0.9, 0.1, 0.0)]


In [687]:
#File Clean-up
downloadedAssets = ['AAVE-USD', 'ADA-USD', 'ALGO-USD', 'ATOM-USD', 'BAL-USD', 'BAND-USD', 'BCH-USD', 'BNT-USD', 'BTC-USD', 'CGLD-USD', 'COMP-USD', 'DASH-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'FIL-USD', 'GRT-USD', 'KNC-USD', 'LINK-USD', 'LRC-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NMR-USD', 'NU-USD', 'OMG-USD', 'OXT-USD', 'REN-USD', 'REP-USD', 'SKL-USD', 'SNX-USD', 'SUSHI-USD', 'UMA-USD', 'UNI-USD','WBTC-USD', 'XLM-USD', 'XTZ-USD', 'YFI-USD', 'ZEC-USD', 'ZRX-USD']
for asset in downloadedAssets:
    for file in listdir('../candlesDataBase/' + asset):
        if file[-3:] == 'pkl':
            with open('../candlesDataBase/' + asset + '/' + file, 'rb') as f:
                data = pkl.load(f)
            if len(data) == 0:
                print('rm ../candlesDataBase/' + asset + '/' + file)
                os.system('rm ../candlesDataBase/' + asset + '/' + file)
            else:
                print('mv ../candlesDataBase/' + asset + '/' + file + ' ../candlesDataBase/' + asset + '/' + asset + "_" + str(int(datetime.datetime.timestamp(data.index[0]))) + "_" + str(int(datetime.datetime.timestamp(data.index[-1]))) + "_" + file.split("_")[-1])
                os.system('mv ../candlesDataBase/' + asset + '/' + file + ' ../candlesDataBase/' + asset + '/' + asset + "_" + str(int(datetime.datetime.timestamp(data.index[0]))) + "_" + str(int(datetime.datetime.timestamp(data.index[-1]))) + "_" + file.split("_")[-1])
            

mv ../candlesDataBase/AAVE-USD/AAVE-USD_1612205820_1616957760_60.pkl ../candlesDataBase/AAVE-USD/AAVE-USD_1612205820_1616957760_60.pkl
mv ../candlesDataBase/AAVE-USD/AAVE-USD_1608052380_1609613760_60.pkl ../candlesDataBase/AAVE-USD/AAVE-USD_1608052380_1609613760_60.pkl
mv ../candlesDataBase/AAVE-USD/AAVE-USD_1609613820_1612205760_60.pkl ../candlesDataBase/AAVE-USD/AAVE-USD_1609613820_1612205760_60.pkl
mv ../candlesDataBase/ADA-USD/ADA-USD_1616083500_1616957760_60.pkl ../candlesDataBase/ADA-USD/ADA-USD_1616083500_1616957760_60.pkl
mv ../candlesDataBase/ALGO-USD/ALGO-USD_1573326240_1575917040_60.pkl ../candlesDataBase/ALGO-USD/ALGO-USD_1573326240_1575917040_60.pkl
mv ../candlesDataBase/ALGO-USD/ALGO-USD_1575918240_1578508980_60.pkl ../candlesDataBase/ALGO-USD/ALGO-USD_1575918240_1578508980_60.pkl
mv ../candlesDataBase/ALGO-USD/ALGO-USD_1609613820_1612205760_60.pkl ../candlesDataBase/ALGO-USD/ALGO-USD_1609613820_1612205760_60.pkl
mv ../candlesDataBase/ALGO-USD/ALGO-USD_1591470300_15940614